# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [3]:
# import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine
import os
import sys
import argparse
import re
import glob

import nltk
nltk.data.path.append('c:\\nltk_data')
nltk.download('punkt_tab', download_dir='c:\\nltk_data')
nltk.download('wordnet', download_dir='c:\\nltk_data')
nltk.download('stopwords', download_dir='c:\\nltk_data')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


import pickle
import joblib




[nltk_data] Downloading package punkt_tab to c:\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.
[nltk_data] Downloading package wordnet to c:\nltk_data...
[nltk_data] Downloading package stopwords to c:\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [4]:
# load data from database
engine = create_engine("sqlite:///./data/DisasterMessages.db")
df = pd.read_sql('SELECT * FROM cleaned_messages', engine)
df.head()


,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
# split data into X and Y
X = df['message']
Y = df.drop(columns=['id', 'message', 'original', 'genre'])

In [34]:
# Category names
category_names = Y.columns.tolist()
print(category_names)

['related', 'request', 'offer', 'aid_related', 'medical_help', 'medical_products', 'search_and_rescue', 'security', 'military', 'child_alone', 'water', 'food', 'shelter', 'clothing', 'money', 'missing_people', 'refugees', 'death', 'other_aid', 'infrastructure_related', 'transport', 'buildings', 'electricity', 'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure', 'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold', 'other_weather', 'direct_report']


### 2. Write a tokenization function to process your text data

In [7]:
def tokenize(text):

    # Remove punctuation characters
    text = re.sub(r"[^a-zA-Z0-9]", " ", text)
    
    # Tokenize text
    tokens = word_tokenize(text)

    # Remove stop words
    tokens = [word for word in tokens if word not in stopwords.words("english")]

    # Lemmatize verbs by specifying pos
    
    lemmatizer = WordNetLemmatizer()
    clean_tokens = [lemmatizer.lemmatize(tok, pos='v').lower().strip() for tok in tokens] 
    
    return clean_tokens



In [8]:
# Test tokenize function
print(X[0])
print(tokenize(X[0]))

Weather update - a cold front from Cuba that could pass over Haiti
['weather', 'update', 'cold', 'front', 'cuba', 'could', 'pass', 'haiti']


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [10]:
# Build a machine learning pipeline

pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier(random_state=42)))
])



### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [11]:
# Split data into train and test sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Train pipeline
pipeline.fit(X_train, Y_train)




c:\Users\samue\miniconda3\envs\udacity\lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x0000017A60BB09D0>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier(random_state=42)))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [12]:
# Predict on test data
Y_pred = pipeline.predict(X_test)


In [13]:
# Report the f1 score, precision and recall for each output category of the dataset

for i, col in enumerate(Y_test):
    print(col)
    print(classification_report(Y_test[col], Y_pred[:, i], zero_division=0))

related
              precision    recall  f1-score   support

           0       0.69      0.40      0.51      1266
           1       0.83      0.94      0.88      3938
           2       0.53      0.50      0.51        40

    accuracy                           0.81      5244
   macro avg       0.68      0.61      0.63      5244
weighted avg       0.79      0.81      0.79      5244

request
              precision    recall  f1-score   support

           0       0.90      0.98      0.94      4349
           1       0.83      0.47      0.60       895

    accuracy                           0.89      5244
   macro avg       0.86      0.73      0.77      5244
weighted avg       0.89      0.89      0.88      5244

offer


c:\Users\samue\miniconda3\envs\udacity\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\samue\miniconda3\envs\udacity\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\samue\miniconda3\envs\udacity\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5218
           1       0.00      0.00      0.00        26

    accuracy                           1.00      5244
   macro avg       0.50      0.50      0.50      5244
weighted avg       0.99      1.00      0.99      5244

aid_related
              precision    recall  f1-score   support

           0       0.80      0.85      0.82      3113
           1       0.76      0.70      0.72      2131

    accuracy                           0.79      5244
   macro avg       0.78      0.77      0.77      5244
weighted avg       0.78      0.79      0.78      5244

medical_help
              precision    recall  f1-score   support

           0       0.93      1.00      0.96      4822
           1       0.71      0.09      0.16       422

    accuracy                           0.92      5244
   macro avg       0.82      0.54      0.56      5244
weighted avg       0.91      0.92      0.90      

c:\Users\samue\miniconda3\envs\udacity\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\samue\miniconda3\envs\udacity\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\samue\miniconda3\envs\udacity\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result)

              precision    recall  f1-score   support

           0       0.96      1.00      0.98      4833
           1       0.91      0.50      0.65       411

    accuracy                           0.96      5244
   macro avg       0.93      0.75      0.81      5244
weighted avg       0.96      0.96      0.95      5244

storm
              precision    recall  f1-score   support

           0       0.95      0.98      0.97      4758
           1       0.74      0.47      0.58       486

    accuracy                           0.94      5244
   macro avg       0.85      0.73      0.77      5244
weighted avg       0.93      0.94      0.93      5244

fire
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      5191
           1       0.00      0.00      0.00        53

    accuracy                           0.99      5244
   macro avg       0.49      0.50      0.50      5244
weighted avg       0.98      0.99      0.98      5244

earthqua

### 6. Improve your model
Use grid search to find better parameters. 

In [14]:
pipeline.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'vect', 'tfidf', 'clf', 'vect__analyzer', 'vect__binary', 'vect__decode_error', 'vect__dtype', 'vect__encoding', 'vect__input', 'vect__lowercase', 'vect__max_df', 'vect__max_features', 'vect__min_df', 'vect__ngram_range', 'vect__preprocessor', 'vect__stop_words', 'vect__strip_accents', 'vect__token_pattern', 'vect__tokenizer', 'vect__vocabulary', 'tfidf__norm', 'tfidf__smooth_idf', 'tfidf__sublinear_tf', 'tfidf__use_idf', 'clf__estimator__bootstrap', 'clf__estimator__ccp_alpha', 'clf__estimator__class_weight', 'clf__estimator__criterion', 'clf__estimator__max_depth', 'clf__estimator__max_features', 'clf__estimator__max_leaf_nodes', 'clf__estimator__max_samples', 'clf__estimator__min_impurity_decrease', 'clf__estimator__min_samples_leaf', 'clf__estimator__min_samples_split', 'clf__estimator__min_weight_fraction_leaf', 'clf__estimator__monotonic_cst', 'clf__estimator__n_estimators', 'clf__estimator__n_jobs', 'clf__estimator__oob_score', 'clf__esti

In [32]:
# Use grid search to find better parameters.

parameters = {
    'vect__ngram_range': ((1, 1), (1, 2)),
    'clf__estimator__max_depth': [10, 50, None],
    'tfidf__use_idf': (True, False)
}

cv = GridSearchCV(pipeline, param_grid=parameters)

In [33]:
cv.fit(X_train, Y_train)

c:\Users\samue\miniconda3\envs\udacity\lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
c:\Users\samue\miniconda3\envs\udacity\lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
c:\Users\samue\miniconda3\envs\udacity\lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
c:\Users\samue\miniconda3\envs\udacity\lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
c:\Users\samue\miniconda3\envs\udacity\lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is no

GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x0000017A60BB09D0>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier(random_state=42)))]),
             param_grid={'clf__estimator__max_depth': [10, 50, None],
                         'tfidf__use_idf': (True, False),
                         'vect__ngram_range': ((1, 1), (1, 2))})

In [35]:
# The best parameters
cv.best_params_

{'clf__estimator__max_depth': None,
 'tfidf__use_idf': True,
 'vect__ngram_range': (1, 2)}

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [36]:
# Predict on test data
Y_pred_cv = cv.predict(X_test)

In [38]:
# Report the f1 score, precision and recall for each output category of the dataset

for i, category in enumerate(category_names):
    print(f"Category: {category}")
    print(classification_report(Y_test[category], Y_pred_cv[:, i], zero_division=0))
    print("\n")

Category: related
              precision    recall  f1-score   support

           0       0.68      0.46      0.55      1266
           1       0.84      0.93      0.88      3938
           2       0.39      0.33      0.36        40

    accuracy                           0.81      5244
   macro avg       0.64      0.57      0.59      5244
weighted avg       0.80      0.81      0.80      5244



Category: request
              precision    recall  f1-score   support

           0       0.90      0.98      0.94      4349
           1       0.84      0.47      0.61       895

    accuracy                           0.89      5244
   macro avg       0.87      0.73      0.77      5244
weighted avg       0.89      0.89      0.88      5244



Category: offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5218
           1       0.00      0.00      0.00        26

    accuracy                           1.00      5244
   macro avg      

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
# Try to improve the model further by using other machine learning algorithms and other features besides the TF-IDF

# Support Vector Machines (SVM)
# SVMs are effective for high-dimensional spaces and are often used for text classification tasks.

pipeline_svm = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(SVC(random_state=42)))
])

cv_svm = GridSearchCV(pipeline_svm, param_grid=parameters)

In [ ]:
# Train pipeline
cv_svm.fit(X_train, Y_train)

In [ ]:
# Using Gradient Boosting Classifier as an alternative to Random Forest Classifier


pipeline_gbc = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(GradientBoostingClassifier(random_state=42)))
])

cv_gbc = GridSearchCV(pipeline_gbc, param_grid=parameters)

In [ ]:
# Train pipeline
cv_gbc.fit(X_train, Y_train)

### 9. Export your model as a pickle file

In [39]:
# EXPORT MODEL AS A PICKLE FILE
model_filepath = './models/model.pkl'
best_model = cv.best_estimator_
pickle.dump(best_model, open(model_filepath, 'wb'))



### 10. Use this notebook to complete `train_classifier.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.